In [ ]:
# Install dependencies (Run this only in Google Colab)
!pip install flask-ngrok gradio scikit-learn requests

# Import necessary libraries
import re
import requests
import gradio as gr
from flask import Flask
from flask_ngrok import run_with_ngrok
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

# Flask app setup
app = Flask(__name__)
run_with_ngrok(app)  # Enables public access to Flask in Colab

# Email validation function
def validate_email(email):
    email_pattern = r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"
    if not re.match(email_pattern, email):
        return "❌ Invalid Email Format", None

    domain = email.split("@")[-1]
    url = f"https://{domain}"

    try:
        response = requests.get(url, timeout=5)
        if response.status_code == 200:
            return f"✅ Valid email. Website ({url}) is reachable.", email
    except requests.exceptions.RequestException:
        pass

    return f"⚠️ Valid email, but website ({url}) is NOT reachable.", email

# Spam classification function
def classify_spam(email_content, model_name):
    email_vector = tfidf.transform([email_content]).toarray()

    if model_name == "SVM":
        prediction = svm_model.predict(email_vector)
    elif model_name == "Decision Tree":
        prediction = dt_model.predict(email_vector)
    elif model_name == "Random Forest":
        prediction = rf_model.predict(email_vector)
    elif model_name == "Naïve Bayes":
        prediction = nb_model.predict(email_vector)
    else:
        return "⚠️ Invalid Model Selection"

    return "🛑 Spam" if prediction == 1 else "✅ Ham"

# Load and train models
tfidf = TfidfVectorizer(max_features=3000)
emails = ["Win a lottery now!", "Hello, how are you?", "Exclusive deal, click here!", "Meeting at 5 PM."]
labels = [1, 0, 1, 0]  # 1 = Spam, 0 = Ham

X = tfidf.fit_transform(emails)

svm_model = SVC()
svm_model.fit(X, labels)

dt_model = DecisionTreeClassifier()
dt_model.fit(X, labels)

rf_model = RandomForestClassifier(n_estimators=50)
rf_model.fit(X, labels)

nb_model = MultinomialNB()
nb_model.fit(X, labels)

# Gradio Interface
def email_page(email):
    validation_message, validated_email = validate_email(email)
    if validated_email is None:
        return validation_message, gr.update(visible=False), gr.update(visible=False), gr.update(visible=False)
    return validation_message, gr.update(visible=True), gr.update(visible=True), gr.update(visible=True)

def spam_page(email_content, model_name):
    return classify_spam(email_content, model_name)

# Gradio UI
with gr.Blocks() as app_ui:
    gr.Markdown("## 📧 Email Spam Checker")

    email_input = gr.Textbox(label="Enter Email ID", placeholder="Enter your email address")
    email_output = gr.HTML()
    content_input = gr.Textbox(label="Email Content", placeholder="Enter email content...", visible=False)
    model_dropdown = gr.Dropdown(["SVM", "Decision Tree", "Random Forest", "Naïve Bayes"], label="Choose Model", visible=False)
    check_button = gr.Button("Check Spam", visible=False)
    spam_result = gr.Textbox(label="Spam Check Result", interactive=False)

    email_input.change(email_page, inputs=email_input, outputs=[email_output, content_input, model_dropdown, check_button])
    check_button.click(spam_page, inputs=[content_input, model_dropdown], outputs=spam_result)

@app.route("/")
def home():
    return "Flask server is running! Open the Gradio link to use the app."

# Run Flask and Gradio
if __name__ == "__main__":
    app_ui.launch(share=True)  # Generates a public Gradio link
    app.run()
